In [35]:
from abc import ABC

import instructor
from litellm import completion
from pydantic import BaseModel


class AbstractLLM(ABC):
    """
    AbstractLLM is an abstract base class for all LLM models.

    Methods:
        - use(self, question: str, response_model: BaseModel) -> BaseModel: Use the LLM model.
    """

    def __init__(self):
        """
        Initialize the LLM model.

        Args:
            kwargs: Dict: The kwargs.

        Returns:
            None
        """
        pass

    def use(self, question: str, response_model: BaseModel) -> BaseModel:
        """
        Calls the LLM model with the given question and response model.

        Args:
            question: str: The question to use the LLM model.
            response_model: BaseModel: The response model.

        Returns:
            response: BaseModel: The response from the LLM model.
        """
        pass


class LLM(AbstractLLM):
    def __init__(self, kwargs):
        """
        Initialize the LLM model.

        Args:
            kwargs: Dict: The kwargs.

        Returns:
            None
        """
        self.model_id = kwargs["model_id"]
        self.temperature = float(kwargs["temperature"])
        self.client = self.init_llm()

    def init_llm(self):
        """
        Initialize the LLM model.

        Returns:
            client: instructor.from_litellm: The LLM model.
        """
        client = instructor.from_litellm(completion)
        return client

    def use(self, question: str, response_model: BaseModel) -> BaseModel:
        """
        Calls the LLM model with the given question and response model.

        Args:
            question: str: The question to use the LLM model.
            response_model: BaseModel: The response model.

        Returns:
            response: BaseModel: The response from the LLM model.
        """
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": question},
        ]

        response = self.client.chat.completions.create(
            messages=messages,
            response_model=response_model,
            model=self.model_id,
            temperature=self.temperature,
        )

        return response

In [48]:
llm = LLM(
    kwargs={"model_id": "claude-sonnet-4-20250514", "temperature": 0.0},
)

In [49]:
class LLMResponseModel(BaseModel):
    chain_of_thought: str
    response: str

In [50]:
response = llm.use("how are you?", LLMResponseModel)

In [51]:
response

LLMResponseModel(chain_of_thought='The user is asking a simple greeting question "how are you?" This is a casual, friendly inquiry about my well-being or current state. I should respond in a warm, helpful manner while being honest about my nature as an AI assistant.', response="I'm doing well, thank you for asking! I'm here and ready to help you with whatever questions or tasks you might have. How are you doing today? Is there anything I can assist you with?")